In [13]:
from dotenv import load_dotenv

_ = load_dotenv()

In [14]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_community.tools.tavily_search import TavilySearchResults

from langchain_google_genai import GoogleGenerativeAI
from langchain_groq import ChatGroq

In [4]:
tool = TavilySearchResults(max_results=2)

In [6]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

In [8]:
!pip install langgraph_checkpoint_sqlite


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
from langgraph.checkpoint.sqlite import SqliteSaver

memory = SqliteSaver.from_conn_string(":memory:")

In [28]:
class Agent:
    def __init__(self, model, tools, checkpointer, system=""):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges("llm", self.exists_action, {True: "action", False: END})
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile(checkpointer=checkpointer)
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def call_openai(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}

In [29]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""
model = ChatGroq(model="llama-3.1-8b-instant")


In [42]:
messages = [HumanMessage(content="What is the weather in sf?")]

In [43]:
thread = {"configurable": {"thread_id": "1"}}

In [44]:
with SqliteSaver.from_conn_string(":memory:") as checkpointer:
    abot = Agent(model, [tool], system=prompt, checkpointer=checkpointer)
    response  = abot.graph.invoke( input={"messages": messages} , config=thread)
    print(response['messages'][-1].content)
    messages = [HumanMessage(content="What about  kenitra ?")]
    response  = abot.graph.invoke( input={"messages": messages} , config=thread)
    print(response['messages'][-1].content)

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'sf weather'}, 'id': 'call_0ppq', 'type': 'tool_call'}
Back to the model!
Based on the search results, the current weather in San Francisco is partly cloudy with a temperature of 54°F (12.2°C).
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'kenitra weather'}, 'id': 'call_p69a', 'type': 'tool_call'}
Back to the model!
Based on the search results, the current weather in Kenitra, Morocco is partly cloudy with a temperature of 65.1°F (18.4°C).
